In [1]:
!pip3 install transformers
!pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 4.1 MB/s eta 0:00:00


# New Section

In [41]:
import json
import re
from transformers import pipeline
import operator
import pandas as pd
import tensorflow as tf
import transformers

from datasets import load_dataset
from tensorflow import keras
from transformers import AutoTokenizer
from transformers import TFAutoModel

from tensorflow.keras import layers
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import train_test_split

In [3]:
f = open("training_task1.txt", "r")
lines = f.read()

lines = lines.replace("}{", "},{")
lines = json.loads(lines)

# Lines is a list of dictionaries, where each dictionary is a single sentence with its labels.


In [4]:
sentences = []
labels = []


for sentence in lines:
    sentences.append(sentence["text"].replace("\n", " "))
    labels.append(sentence["labels"])


In [5]:
list_of_labels = {}

for label in labels:
  if len(label) == 0:
      label.append('none')
  for sub_label in label:
    if ''.join(sub_label) not in list_of_labels.keys():
      str_label = ''.join(sub_label).lower()
      str_label = re.sub(r"[^a-zA-Z0-9]+", ' ', str_label)
      if str_label not in list_of_labels.keys():
        list_of_labels[str_label] = len(list_of_labels)

print(list_of_labels)

{'black and white fallacy dictatorship': 0, 'none': 1, 'loaded language': 2, 'name calling labeling': 3, 'slogans': 4, 'smears': 5, 'causal oversimplification': 6, 'appeal to fear prejudice': 7, 'exaggeration minimisation': 8, 'reductio ad hitlerum': 9, 'repetition': 10, 'glittering generalities virtue ': 11, 'misrepresentation of someone s position straw man ': 12, 'doubt': 13, 'obfuscation intentional vagueness confusion': 14, 'whataboutism': 15, 'flag waving': 16, 'thought terminating clich ': 17, 'presenting irrelevant data red herring ': 18, 'appeal to authority': 19, 'bandwagon': 20}


In [6]:
print(labels[2])

['Loaded Language', 'Name calling/Labeling', 'Slogans', 'Smears']


In [7]:
class Dataset:
    def __init__(self, sentences, labels):
        self.sentences = sentences
        self.labels = labels

    @staticmethod
    def preprocess(sentences, labels):

        preprocessed_sentences = []
        preprocessed_labels = []
        numbered_labels = []

        for sentence in sentences:
            sentence = sentence.lower()
            sentence = re.sub(r"[^a-zA-Z0-9]+", ' ', sentence)
            sentence = re.sub(r"http\S+", "", sentence)
            sentence = re.sub(r"www\S+", "", sentence)
            preprocessed_sentences.append(sentence)

        for i in range(len(labels)):
            label = labels[i]
            list_of_labels_for_a_sent = []
            numbered_labels_for_a_sent = []
            for j in range(len(label)):
                label[j] = label[j].lower()
                label[j] = re.sub(r"[^a-zA-Z0-9]+", ' ', label[j])
                list_of_labels_for_a_sent.append(label[j].split(","))
                temp_label_in_string = ' '.join(label[j].split(","))
                numbered_labels_for_a_sent.append([list_of_labels[temp_label_in_string]])



            preprocessed_labels.append(list_of_labels_for_a_sent)
            numbered_labels.append(numbered_labels_for_a_sent)

            print(list_of_labels_for_a_sent)
            print(numbered_labels_for_a_sent)

        return preprocessed_sentences, numbered_labels

In [8]:
cleaned_sentences, cleaned_labels = Dataset.preprocess(sentences, labels)

# cleaned sentences contain the list of preprocessed sentences
# cleaned labels contain the list of preprocessed labels for corresponding sentences (list of lists for each sentence.)
'''
Eg:
our elders were called to war to save lives we are being called to sit on the couch to save theirs we can do this
[['appeal to fear prejudice'], ['causal oversimplification'], ['exaggeration minimisation'], ['slogans']]
'''

[['black and white fallacy dictatorship']]
[[0]]
[['none']]
[[1]]
[['loaded language'], ['name calling labeling'], ['slogans'], ['smears']]
[[2], [3], [4], [5]]
[['causal oversimplification'], ['loaded language'], ['name calling labeling'], ['slogans'], ['smears']]
[[6], [2], [3], [4], [5]]
[['none']]
[[1]]
[['appeal to fear prejudice'], ['causal oversimplification'], ['exaggeration minimisation'], ['slogans']]
[[7], [6], [8], [4]]
[['loaded language'], ['name calling labeling'], ['reductio ad hitlerum'], ['smears']]
[[2], [3], [9], [5]]
[['loaded language'], ['name calling labeling'], ['smears']]
[[2], [3], [5]]
[['loaded language'], ['name calling labeling'], ['repetition']]
[[2], [3], [10]]
[['causal oversimplification'], ['glittering generalities virtue '], ['loaded language'], ['name calling labeling']]
[[6], [11], [2], [3]]
[['appeal to fear prejudice'], ['loaded language'], ['slogans'], ['smears']]
[[7], [2], [4], [5]]
[['causal oversimplification'], ['loaded language'], ['name 

"\nEg:\nour elders were called to war to save lives we are being called to sit on the couch to save theirs we can do this\n[['appeal to fear prejudice'], ['causal oversimplification'], ['exaggeration minimisation'], ['slogans']]\n"

In [9]:
print(cleaned_sentences[3])
print(cleaned_labels[3])

pathetic the cowardly asshole weak failure impeach it again 45 the coward s afraid of strong women politicususa com childish trump won t meet with pelosi on coronavirus because he doesn t like her 
[[6], [2], [3], [4], [5]]


In [10]:
model_name = 'bert-base-uncased'
model = TFAutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [11]:
text = cleaned_sentences[0]
print(text)
tokens = tokenizer(text)
print(tokens)

there are only two genders female male 
{'input_ids': [101, 2045, 2024, 2069, 2048, 5907, 2015, 2931, 3287, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [12]:
print(cleaned_sentences)
print(cleaned_labels)

['there are only two genders female male ', 'this is not an accident ', 'so bernie bros haven t committed violence eh power comes from the barrel of a gun comrades what about the one who shot congressman scalise or the dayton ohio mass shooter ', 'pathetic the cowardly asshole weak failure impeach it again 45 the coward s afraid of strong women politicususa com childish trump won t meet with pelosi on coronavirus because he doesn t like her ', 'who trump represents who democrats represent ', 'our elders were called to war to save lives we are being called to sit on the couch to save theirs we can do this ', 'adolf hitler believed state power would fix germany proposed gun control promised a political revolution called himself socialist blamed jews for germany s problems wanted a heavily regulated economy bernie sanders believes state power will fix america proposes gun control promises a political revolution calls himself socialist blames the rich for america s problems wants a heavily

In [13]:
print(len(cleaned_sentences))

# Including the <s> and </s> tokens, the max sentence length = 82
# There are in total 688 sentences

688


In [70]:
inputs = tokenizer(cleaned_sentences,padding=True,truncation=True,return_tensors='tf')
print(inputs)

{'input_ids': <tf.Tensor: shape=(688, 82), dtype=int32, numpy=
array([[  101,  2045,  2024, ...,     0,     0,     0],
       [  101,  2023,  2003, ...,     0,     0,     0],
       [  101,  2061, 15941, ...,     0,     0,     0],
       ...,
       [  101,  1045,  2490, ...,     0,     0,     0],
       [  101, 21877, 10483, ...,     0,     0,     0],
       [  101,  2065,  2017, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(688, 82), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(688, 82), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

***BERT is a encoder only transformer, now we feed our tokenized sentences to BERT and use the encoder hidden state and input it to the decoder.***

In [71]:
outputs = model(inputs)
print(outputs)

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(688, 82, 768), dtype=float32, numpy=
array([[[-3.78582835e-01,  1.74619019e-01, -1.12571716e-01, ...,
         -2.18954861e-01,  3.68135542e-01,  4.78350252e-01],
        [ 2.34085694e-03,  4.21275973e-01, -4.79490161e-01, ...,
         -2.94536442e-01,  4.60918993e-01, -4.60693613e-02],
        [ 2.39521578e-01,  1.24723732e-01, -1.03663921e-01, ...,
         -4.30854976e-01, -1.71649545e-01,  1.24428391e-01],
        ...,
        [-2.68685341e-01, -4.94127452e-01,  8.20762515e-02, ...,
         -8.45117792e-02, -5.51631078e-02,  3.67361039e-01],
        [ 1.36672154e-01,  1.34365320e-01,  1.98123172e-01, ...,
         -1.13645181e-01,  1.92668021e-01,  2.49377653e-01],
        [-7.82009587e-03, -2.32678190e-01,  2.25917488e-01, ...,
         -7.60363415e-02, -9.11264867e-02,  2.47277632e-01]],

       [[ 1.08501874e-01,  2.37600267e-01, -1.83187962e-01, ...,
         -1.37840182e-01,  1.87398762e-01, 

**last_hidden_state**=<tf.Tensor: *shape=(688, 82, 768)* this represents each tokens' (82 = max sentence length) hidden state of each sentence (688 = total no of sentences) and the size of the hidden state = 768.

**pooler_output**=<tf.Tensor: *shape=(688, 768)* this representes the ONE final encoder representation produced by BERT of hidden dimension 768.

Now which one to use? last_hidden_state or pooler_output? for a task like text classification we do not need the hidden states of each word, so we can neglect the last_hidden_state

In [17]:
print(outputs.pooler_output)

tf.Tensor(
[[-0.87234765 -0.3967047  -0.85567886 ... -0.5996428  -0.60494167
   0.8361169 ]
 [-0.88086766 -0.2323565   0.1594327  ...  0.19720143 -0.5746141
   0.9110737 ]
 [-0.8037872  -0.4983936  -0.8424049  ... -0.66819364 -0.6176859
   0.70863134]
 ...
 [-0.86765814 -0.4763081  -0.8485897  ... -0.62210035 -0.5992428
   0.87038606]
 [-0.8027915  -0.4714605  -0.9587117  ... -0.8269426  -0.62785774
   0.6685048 ]
 [-0.7488531  -0.46447763 -0.79031867 ... -0.60748297 -0.59233946
   0.78106546]], shape=(688, 768), dtype=float32)


In [39]:
import numpy as np

array_size = (688, 21)
labels_array = np.zeros(array_size)
print(labels_array)
i = 0
for label in cleaned_labels:
  for sub_label in label:
    for number in sub_label:
      labels_array[i][number] = 1
  i += 1

print(labels_array)
label_tensor = tf.convert_to_tensor(labels_array)

print(label_tensor.shape)




[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
(688, 21)


Building the FFN using the BERT encodings and labels


In [93]:
NUM_UNITS = 512
NUM_UNITS2 = 128
NUM_CLASSES = 21

ffn = keras.Sequential([
  layers.Dense(NUM_UNITS, activation='relu'),
  layers.Dense(NUM_UNITS2, activation='relu'),
  layers.Dense(NUM_CLASSES, activation='softmax')
])

In [94]:
ffn.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])


In [74]:
print(outputs['pooler_output'].shape)

(688, 768)


In [101]:
NUM_EPOCHS =  500
BATCH_SIZE = 128


train_loop = ffn.fit(outputs['pooler_output'] ,label_tensor, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/500
6/6 [==============================] - 0s 10ms/step - loss: 0.0017 - accuracy: 0.7616
Epoch 2/500
6/6 [==============================] - 0s 7ms/step - loss: 0.0015 - accuracy: 0.7907
Epoch 3/500
6/6 [==============================] - 0s 6ms/step - loss: 0.0015 - accuracy: 0.7515
Epoch 4/500
6/6 [==============================] - 0s 8ms/step - loss: 0.0015 - accuracy: 0.7820
Epoch 5/500
6/6 [==============================] - 0s 6ms/step - loss: 0.0015 - accuracy: 0.7631
Epoch 6/500
6/6 [==============================] - 0s 6ms/step - loss: 0.0014 - accuracy: 0.7544
Epoch 7/500
6/6 [==============================] - 0s 6ms/step - loss: 0.0017 - accuracy: 0.7733
Epoch 8/500
6/6 [==============================] - 0s 7ms/step - loss: 0.0017 - accuracy: 0.7878
Epoch 9/500
6/6 [==============================] - 0s 7ms/step - loss: 0.0017 - accuracy: 0.7689
Epoch 10/500
6/6 [==============================] - 0s 7ms/step - loss: 0.0014 - accuracy: 0.7660
Epoch 11/500
6/6 [==========

In [102]:
saved_model = ffn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               393728    
                                                                 
 dense_13 (Dense)            (None, 128)               65664     
                                                                 
 dense_14 (Dense)            (None, 21)                2709      
                                                                 
Total params: 462,101
Trainable params: 462,101
Non-trainable params: 0
_________________________________________________________________
